# Figure 5
- maybe something related to off-periods local sleep

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from neuropy import plotting
from neuropy.utils import signal_process
import subjects
import pandas as pd
from neuropy.analyses import ExplainedVariance

sd_sess = (
    subjects.sd.ratNday1
    + subjects.sd.ratSday3
    + subjects.sd.ratRday2
    + subjects.sd.ratUday4
)
nsd_sess = subjects.nsd.ratNday2+subjects.nsd.ratSday2+subjects.nsd.ratUday2